Importing modules

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from Layer import *
from network import Network
from activation_func import tanh, tanh_prime, sigmoid, sigmoid_prime, softmax, softmax_prime, relu, relu_prime
from loss_func import mse, mse_prime, cross_entropy, cross_entropy_prime
from data_func import vectorize_labels, k_fold
from confusion_matrix import plot_confusion_matrix

Importing data

In [2]:
# import data
dataset = np.load("../dataset.npz")
training = dataset['arr_0'][:6000]  # training_img
labels = vectorize_labels(dataset['arr_2'][:6000])  # training_labels
test = dataset['arr_1']  # test_img
original_test_labels = dataset['arr_3']  # test_labels
test_labels = vectorize_labels(original_test_labels)
np.random.seed(10)

# normalize data
training /= 255
test /= 255

Setting configuration

In [5]:
# hyper parameters
learning_rate = 5e-3
hidden_layers = [300, 200, 100]
epochs = 10
batch_size = 32
weight_decay = 0.01

# functions
activation = relu
activation_prime = relu_prime
loss = cross_entropy
loss_prime = cross_entropy_prime
loss_activation = softmax
loss_activation_prime = softmax_prime

# test size
test_size = 10000

# specify input and output parameters
features = 784
output_classes = 10

Train the network

In [6]:
# set up the network with specified layers, loss, and activation
net = Network()
net.setup_net(hidden_layers, activation, features, output_classes,
              activation_prime,
              loss_activation, loss_activation_prime,
              loss, loss_prime,
              FCLayer, ActivationLayer, LossLayer)

# prepare data for training
fold_train_data, fold_train_labels, fold_val_data, fold_val_labels = k_fold(training, labels, 5, 5)

# train the model on training data and labels using specific hyper-parameters
errors, val_errors = net.fit(fold_train_data, fold_train_labels, fold_val_data, fold_val_labels,
                             epochs=epochs, learning_rate=learning_rate, batch_size=batch_size, momentum=True, weight_decay=weight_decay)

# print the accuracy
print("The test accuracy of the network is: {}"
      .format(
    net.accuracy(x=test[:test_size], y_true=original_test_labels[:test_size], errors=errors, val_errors=val_errors)))


epoch 1/10   training error=4.617074  validation error=0.935300 validation accuracy=0.726667
epoch 2/10   training error=0.624157  validation error=0.647859 validation accuracy=0.825000
epoch 3/10   training error=0.398579  validation error=0.578385 validation accuracy=0.834167
epoch 4/10   training error=0.282573  validation error=0.499965 validation accuracy=0.865000
epoch 5/10   training error=0.212001  validation error=0.480768 validation accuracy=0.880000
epoch 6/10   training error=0.166453  validation error=0.421059 validation accuracy=0.892500
epoch 7/10   training error=0.128018  validation error=0.437889 validation accuracy=0.898333
epoch 8/10   training error=0.097608  validation error=0.435638 validation accuracy=0.894167
epoch 9/10   training error=0.073666  validation error=0.438201 validation accuracy=0.903333
epoch 10/10   training error=0.062203  validation error=0.430668 validation accuracy=0.906667
The test accuracy of the network is: 0.8864


Plot results

In [ ]:
def plot_error(error, val_error):
    fig, ax1 = plt.subplots()
    ax1.plot(error, 'r', label="training loss ({:.6f})".format(error[-1]))
    ax1.plot(val_error, 'b--', label="validation loss ({:.6f})".format(val_error[-1]))
    ax1.grid(True)
    ax1.set_xlabel('iteration')
    ax1.legend(loc="best", fontsize=9)
    ax1.set_ylabel('loss', color='r')
    ax1.tick_params('y', colors='r')
    plt.show()


plot_error(errors, val_errors)